# Finalise facebook posts topics
Finalises the facebook data topic data.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

YEAR_FILTER = 2021
MONTH_FILTER = 5
MONTH_PREFIX = f"year_filter={YEAR_FILTER}/month_filter={MONTH_FILTER}/"

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}{RUN_DATE}/facebook_posts/{MONTH_PREFIX}"

# OUTPUT
FACEBOOK_POSTS_PERSIST = f"s3://buildup-dev-us-tables/facebook_posts/parquet_exports/facebook_posts_topics_v1/{MONTH_PREFIX}{YEAR_FILTER}-{MONTH_FILTER}.parquet"

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
RUN_DATE,
sep='\n',
)

In [ ]:
# %env DASK_CLUSTER_IP=

In [ ]:
utils.dask_global_init()

In [ ]:
facebook_posts_df = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "facebook_posts_final")).dataframe

In [ ]:
facebook_posts_df.head()

In [ ]:
topics = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "topics")).dataframe

In [ ]:
topics.head()

In [ ]:
facebook_posts_topics_final = finalise.join_topics_to_facebook_posts(topics, facebook_posts_df)

In [ ]:
facebook_posts_topics_final.head()

In [ ]:
facebook_posts_topics_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(FACEBOOK_POSTS_PERSIST, facebook_posts_topics_final)

In [ ]:
_ = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "facebook_posts_topics_final"), facebook_posts_topics_final)

In [ ]:
artifacts.dataframes.read_schema(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "facebook_posts_topics_final"))